# Исследование надёжности заёмщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных:**
- *children* — количество детей в семье
- *days_employed* — общий трудовой стаж в днях
- *dob_years* — возраст клиента в годах
- *education* — уровень образования клиента
- *education_id* — идентификатор уровня образования
- *family_status* — семейное положение
- *family_status_id* — идентификатор семейного положения
- *gender* — пол клиента
- *income_type* — тип занятости
- *debt* — имел ли задолженность по возврату кредитов
- *total_income* — ежемесячный доход
- *purpose* — цель получения кредита

**Для выполнения исследования выполним следующие шаги:**
1. Определим и заполним пропущенные значения.
2. Заменим вещественный тип данных на целочисленный.
3. Удалим дубликаты.
4. Выделим леммы в значениях столбца с целями получения кредита.
5. Категоризируем данные.
6. Ответим на вопросы:
 -  Есть ли зависимость между наличием детей и возвратом кредита в срок?
 -  Есть ли зависимость между семейным положением и возвратом кредита в срок?
 -  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
 -  Как разные цели кредита влияют на его возврат в срок?
7. Сформулируем общий вывод.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
# <импортируем библиотеки>
import pandas as pd

from pymystem3 import Mystem
m = Mystem() 
from collections import Counter

from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')

In [2]:
# <чтение файла с данными с сохранением в client>
clients = pd.read_csv('/datasets/data.csv')

In [3]:
# <получение первых 15 строк таблицы client>
clients.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Каждая строка таблицы содержит информацию о заёмщиках банка. Для проверки рабочих гипотез особенно ценны столбцы children, family_status, total_income, purpose и debt. Пропущенные значения встречаются в столбцах days_employed и total income, причем их одинаковое количество. Также в столбце days_employed встречаются отрицательные значения. В столбце education встречаются одинаковые значения разного регистра на что тоже следует обратить внимание.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
# < проверим перечень названий столбцов таблицы clients>
clients.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [6]:
# <суммарное количество пропусков, выявленных методом isnull() в таблице clients>
clients.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
# Проверим совпадают ли пропущенные значения в столбцах days_employed и total_income
#Для этого создадим новую таблицу
null_value = clients[(clients['days_employed'].isnull()) & (clients['total_income'].isnull())]
null_value.info()
#Пропущенные значения совпадают в обеих таблицах. Предпложим, что это связано с ошибкой переноса данных.
#На всякий случай проверим связаны ли пропуски с возрастом или типом занятости 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


In [8]:
#Проверим есть ли зависимость пропусков в столбцах от типа занятости клиентов
income_type = null_value.groupby('income_type')['income_type'].count()
print(income_type)
# в столбце income_type присутствуют различные категории типов занятости, поэтому пропуски в данных не зависят от типа занятости

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: income_type, dtype: int64


In [9]:
#Проверим есть ли зависимость пропусков в столбцах от возраста клиентов
dob_years = null_value.groupby('dob_years')['dob_years'].count()
print(dob_years.head())
# в столбце dob_years есть пропуски совсем в разных возрастных категориях, поэтому они тоже не взаимосвязаны

dob_years
0     10
19     1
20     5
21    18
22    17
Name: dob_years, dtype: int64


In [10]:
# Проверим связаны ли как-нибудь отрицательные значения в столбце days_employed с значениями стобца income_type
days_employed_less_than_zero = clients[clients['days_employed'] < 0]
print(days_employed_less_than_zero.groupby('income_type')['days_employed'].count())
# Отрицательные значения также присутствуют в разных категориях, кроме категории пенсионеров

income_type
в декрете              1
госслужащий         1312
компаньон           4577
предприниматель        1
сотрудник          10014
студент                1
Name: days_employed, dtype: int64


In [11]:
# Скорее всего отрицательные значения в столбце days_employed связаны с ошибкой переноса данных
# Заменим эти значения на положительные. Для этого возьмем этот стобец по модулю
clients['days_employed'] = clients['days_employed'].abs()

In [12]:
#Посчитаем медианный доход в зависимости от типа занятости
#clients_total_income_avg_pivot = clients.pivot_table(index = 'income_type', columns='dob_years', values='total_income', aggfunc='mean')

clients_total_income_median = clients.groupby('income_type')['total_income'].median()
clients_total_income_median
# Получается непонятный  средний доход у безработных и пенсионеров

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [13]:
#Заменим пропущенные значения в столбце total_income на медианные в зависимости от типа занятости
clients['total_income'] = clients['total_income'].fillna(clients.groupby('income_type')['total_income'].transform('median'))

In [14]:
#Посчитаем медианное количество отработанных дней и заменим пропущенные значения в столбце days_employed
clients_days_employed_median = clients.groupby('income_type')['days_employed'].median()
clients_days_employed_median

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [15]:
# Получается предельно высокое среднее количество отработанных дней у безработных и пенсионеров
# Но так как данный стобец нам не пригодится для исследования, заменим отсутствующие значения на медианные по группам типов занятости
clients['days_employed'] = clients['days_employed'].fillna(clients.groupby('income_type')['days_employed'].transform('median'))

In [16]:
#Проверим остались ли пропущенные значения в таблице
clients.info()
#Теперь таблица полностью заполнена

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

На данном этапе мы выявили в каких столбцах находились пропуски и предположили, что они скорее всего связаны с ошибкой переноса данных. Поэтому было решено заменить их на медианные значения, которые мы рассчитали для каждого типа занятости заемщиков.

### Замена типа данных

In [17]:
# Столбец days_employed исчисляется в днях, поэтому приведем его к целому значению
clients['days_employed'] = clients['days_employed'].astype('int')
# Такая точность в столбеце total_income нам не потребуется, поэтому проведем ту же самую операцию
clients['total_income'] = clients['total_income'].astype('int')
clients

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

На данном шаге мы привели столбец days_employed к целочисленным значениям, так как он измеряется в днях.
Столбец total_income также привели к типу int, нам не важна такая точность, а целые числа проще воспринимаются.

### Обработка дубликатов

In [18]:
# Проверим столбец children на количество различных вариантов
print(clients['children'].value_counts())
# Тут встречаются отрицательные значения и явно ошибочные в виде 20 детей

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [19]:
#Заменим значения -1 из столбца children на 1, а значение 20 на 2
#Предположим что -1 и 20 в значениях появились из-за ошибки ввода данных
clients['children'] = clients['children'].replace(-1, 1)
clients['children'] = clients['children'].replace(20, 2)

print(clients['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [20]:
# Проверим столбец education на количество различных вариантов
print(clients['education'].value_counts())
# Тут встречается много одинаковых значений с разным регистром

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


In [21]:
# Переведём столбец education в нижний регистр методом str.lower() и заново посмотрим на него
clients['education'] = clients['education'].str.lower()
print(clients['education'].value_counts())
#Теперь все категории отчетливо видно

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [22]:
# Проверим столбец family_status на количество различных вариантов
print(clients['family_status'].value_counts())
# Тут все в порядке, оставим все категории так как есть

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [23]:
# Проверим столбец gender на количество различных вариантов
print(clients['gender'].value_counts())
# Видим странное значение XNA, чтобы оно нас не смущало удалим его

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [24]:
# Удаляем строку с странным значением пола
clients = clients.loc[clients['gender'] != 'XNA']
print(clients.groupby('gender')['gender'].count())

gender
F    14236
M     7288
Name: gender, dtype: int64


In [25]:
# Наконец проверим нашу таблицу на количество дубликатов
clients.duplicated().sum()
# Видим, что в таблице есть дубликаты. Скорее всего это обычное задвоение данных.

71

In [26]:
# Удалим дубликаты и присвоим строкам новые индексы
clients = clients.drop_duplicates().reset_index(drop = True)

### Вывод

На данном этапе мы обработали все дубликаты. Привели к единому регистру столбец education, удалили непонятное значение из стобца gender, проверили стоблец children и заменили явно неккоректные значения в нём. В столбце family_status все в порядке, оставим его без изменения. В итоге почти все столбцы необходимые нам для исследования приведены в порядок.

### Лемматизация

In [27]:
#Выведем список уникальных значений по столбцу purpose
clients['purpose'].value_counts()
# Просмотрев все уникальные значения выделим самые попуряные цели: свадьба, недвижимость, жилье, автомобиль и образование

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка своего жилья                      620
покупка недвижимости                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [28]:
#Создадим функцию которая будет проверять самые популярные цели в лемматизированном списке и присваивать им новую категорию

def purpose_optimizate(data):
    lemmas = m.lemmatize(data)
    if 'свадьба' in lemmas:
        return 'Кредит на свадьбу'
    if ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'Кредит на недвижимость'
    if 'автомобиль' in lemmas:
        return 'Кредит на автомобиль'
    if 'образование' in lemmas:
        return 'Кредит на образование'
    return 'Неизвестная цель'

#Создадим новый столбец, который заполним новыми категориями

clients['purpose_group'] = clients['purpose'].apply(purpose_optimizate)
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Кредит на недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Кредит на автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Кредит на недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Кредит на образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Кредит на свадьбу


### Вывод

На данном шаге мы выяснили какие ключевые слова чаще всего встречаются в столбце purpose.
Далее мы лемматзировали столбец с целями и написали функцию, которая искала эти ключевые слова в лемматизированном списке и возвращала новую категорию.

### Категоризация данных

In [29]:
#Создадим список в котором будеут определены интервалы дохода, используя разбиение по квантилям данных 
clients['total_income_group'] = pd.qcut(clients['total_income'],5)
rest_dict = clients[['total_income_group']]
rest_dict = rest_dict.drop_duplicates().sort_values('total_income_group').reset_index(drop=True)
rest_dict

,total_income_group
0,"(20666.999, 98536.4]"
1,"(98536.4, 132131.0]"
2,"(132131.0, 161332.0]"
3,"(161332.0, 214622.8]"
4,"(214622.8, 2265604.0]"


In [30]:
#Создадим функцию которая будет проверять доход и присваивать строке категорию
def total_income_group(count):
    if count in rest_dict.loc[0, 'total_income_group']:
        return 'Низкий достаток'
    if count in rest_dict.loc[1, 'total_income_group']:
        return 'Достаток ниже среднего'
    if count in rest_dict.loc[2, 'total_income_group']:
        return 'Средний достаток'
    if count in rest_dict.loc[3, 'total_income_group']:
        return 'Достаток выше среднего'
    return 'Высокий достаток'

#Создадим новый столбец, который заполним новыми категориями

clients['income_status'] = clients['total_income'].apply(total_income_group)
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,total_income_group,income_status
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Кредит на недвижимость,"(214622.8, 2265604.0]",Высокий достаток
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Кредит на автомобиль,"(98536.4, 132131.0]",Достаток ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Кредит на недвижимость,"(132131.0, 161332.0]",Средний достаток
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Кредит на образование,"(214622.8, 2265604.0]",Высокий достаток
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Кредит на свадьбу,"(132131.0, 161332.0]",Средний достаток


In [31]:
# Проверим словарь из столбцов education_id и education. Нет ли там каких-то странных категорий?
rest_dict_education = clients[['education', 'education_id']]
rest_dict_education = rest_dict_education.drop_duplicates().reset_index(drop=True)
rest_dict_education
# Все в порядке. Каждому значению education_id соответсвует одно значение из education

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [32]:
# Проверим словарь из столбцов education и education_id. Нет ли там каких-то странных категорий?
rest_dict_family = clients[['family_status', 'family_status_id']]
rest_dict_family = rest_dict_family.drop_duplicates().reset_index(drop=True)
rest_dict_family
# Все в порядке. Каждому значению family_status_id соответсвует одно значение из family_status

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


### Вывод

На данном этапе мы разделили столбец total_income на категории при помощи квантилей и выделили их в новый столбец.
Также проверили словари family_status - family_status_id и education - education_id на подозрительные значения, но там все оказалось в порядке.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [33]:
# Посмотрим есть ли зависимость между наличием детей и возвратом кредита в срок?
chidren_grouped = clients.groupby('children').agg({'debt': ['count', 'sum']})
chidren_grouped['part'] = chidren_grouped['debt']['sum'] / chidren_grouped['debt']['count']
chidren_grouped.sort_values(by = 'children')

debt            part
          count   sum          
children                       
0         14090  1063  0.075444
1          4855   445  0.091658
2          2128   202  0.094925
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000

### Вывод

Доля заемщиков без детей чаще остальных возвращают кредит в срок, это отчетливо видно по полученным нами результатам. По значениям от 0 до 2 детей можно увидеть такую зависимость, что с прибавлением детей количество просрочек возрастает. Дальше эта зависимость пропадает, но интересно было бы это проверить это еще раз на более большой выборке данных.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
#Проверим есть ли зависимость между семейным положением и возвратом кредита в срок?
family_status_grouped = clients.groupby('family_status').agg({'debt': ['count', 'sum']})
family_status_grouped['part'] = family_status_grouped['debt']['sum'] / family_status_grouped['debt']['count']
family_status_grouped.sort_values(by = 'part')

debt           part
                       count  sum          
family_status                              
вдовец / вдова           959   63  0.065693
в разводе               1195   85  0.071130
женат / замужем        12339  931  0.075452
гражданский брак        4150  388  0.093494
Не женат / не замужем   2810  274  0.097509

### Вывод

Исходя из полученных результатов отчетливо видно, что люди которые были или находятся в браке более добросовестно платят по кредиту. Люди не состоявшие в официальном браке чаще пренебрегают платежами в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
#Изучем есть ли зависимость между семейным положением и возвратом кредита в срок?
income_status_grouped = clients.groupby('income_status').agg({'debt': ['count', 'sum']})
income_status_grouped['part'] = income_status_grouped['debt']['sum'] / income_status_grouped['debt']['count']
income_status_grouped.sort_values(by = 'part')

debt           part
                       count  sum          
income_status                              
Высокий достаток        4291  300  0.069914
Низкий достаток         4291  344  0.080168
Достаток выше среднего  4290  361  0.084149
Достаток ниже среднего  4290  361  0.084149
Средний достаток        4291  375  0.087392

### Вывод

Люди с высоким достатком более ответственно подходят к платежам в срок по кредитам, что в целом логично.
На втором месте идут люди наоборот с наименьшим достатком.
Доля людей находяшихся в промежутке между ними "средний класс" чаще задерживают платежи.

- Как разные цели кредита влияют на его возврат в срок?

In [36]:
#Изучем есть ли зависимость между целями кредита и возвратом кредита в срок?
purpose_group_pivot = clients.pivot_table(index = 'purpose_group', values = 'debt', aggfunc = ['sum', 'count'])
purpose_group_pivot['part'] = purpose_group_pivot['sum'] / purpose_group_pivot['count']
purpose_group_pivot.sort_values(by = 'part')

,sum,count,part
,debt,debt,
purpose_group,,,
Кредит на недвижимость,782,10810,0.072340
Кредит на свадьбу,186,2324,0.080034
Кредит на образование,370,4013,0.092200
Кредит на автомобиль,403,4306,0.093590


### Вывод

Операциями с недвижимостью имееют самую низкую долю просрочки по кредитам. Возможно это связано с ипотечной программой и более низкими процентными ставками. Кредиты связанные с образованием и автомобилями имеют самую большую долю задержек.

### Шаг 4. Общий вывод

Исходя из проведенного исследования можно сделать вывод что зависимость между семейным положением и возвратом кредита в срок наиболее ярко выраженная. Люди, которые находятся или когда-то состояли в браке чаще платят по кредиту вовремя. С наличием детей и ежемесячным доходом всё не так однозначно. Зависимость что чем меньше детей, тем меньше просрочек по кредиту работает для тех у кого от 0 до 2 детей. Для более точного дальнейшего исследования можно было бы рассмотреть выборку данных побольше. Ситуция с ежемесячным доходом следующая: люди с высоким достатком более ответсвенные, чем остальные. Однако заемщики с низким достатком платят в срок чаще чем "средний класс". Также отметим, что клиенты берущие займы на недвижимость или свадьбу платят по кредиту в срок чаще тех, кто использует кредит на автомобили и образование.